In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, ttest_ind
import matplotlib.font_manager as fm
import platform

plt.rcParams['font.family'] = 'Malgun Gothic'  # 또는 'NanumGothic', 'Arial Unicode MS'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

USE_SPEARMAN = False  # True: 스피어만, False: 피어슨
USE_STANDARDIZATION = True  # Z-score 표준화 여부


# 데이터 불러오기
asthma_df = pd.read_excel('Asthma_top30_Sum_removed_outliers.xlsx', index_col=0, parse_dates=True)
dust25_df = pd.read_excel("pm25_top30_fixed.xlsx", index_col=0, parse_dates=True)

# 날짜 동기화
common_index = dust25_df.index.intersection(asthma_df.index)
dust25_df = dust25_df.loc[common_index].sort_index()
asthma_df = asthma_df.loc[common_index].sort_index()

# 숫자 변환 및 결측치 보간
dust25_df = dust25_df.apply(pd.to_numeric, errors='coerce').interpolate().dropna()
asthma_df = asthma_df.apply(pd.to_numeric, errors='coerce').loc[dust25_df.index]

# 선택적으로 표준화
if USE_STANDARDIZATION:
    dust25_df = (dust25_df - dust25_df.mean()) / dust25_df.std()
    asthma_df = (asthma_df - asthma_df.mean()) / asthma_df.std()

# 대도시 / 비대도시 분할
dust_major = dust25_df.iloc[:, 0:8]
dust_minor = dust25_df.iloc[:, 8:]
asthma_major = asthma_df.iloc[:, 0:8]
asthma_minor = asthma_df.iloc[:, 8:]

# 상관계수 함수 (피어슨 or 스피어만 선택)
def get_corrs(df1, df2):
    corrs = []
    for i in range(df1.shape[1]):
        x, y = df1.iloc[:, i], df2.iloc[:, i]
        valid = x.notna() & y.notna()
        if valid.sum() >= 2:
            if USE_SPEARMAN:
                corr, _ = spearmanr(x[valid], y[valid])
            else:
                corr, _ = pearsonr(x[valid], y[valid])
            corrs.append(corr)
        else:
            corrs.append(np.nan)
    return corrs

# 상관계수 추출
major_corrs = get_corrs(dust_major, asthma_major)
minor_corrs = get_corrs(dust_minor, asthma_minor)

# 평균 비교 및 통계 검정
print(f"대도시 평균 상관계수: {np.nanmean(major_corrs):.3f}")
print(f"비대도시 평균 상관계수: {np.nanmean(minor_corrs):.3f}")
t_stat, p_val = ttest_ind(major_corrs, minor_corrs, nan_policy='omit')
print(f"독립표본 t-검정 결과: t = {t_stat:.3f}, p = {p_val:.4f}")

if p_val < 0.05:
    print("대도시와 비대도시 간 상관관계에 유의미한 차이가 존재.")
else:
    print("대도시와 비대도시 간 상관관계에 유의미한 차이 없음.")

대도시 평균 상관계수: 0.520
비대도시 평균 상관계수: 0.408
독립표본 t-검정 결과: t = 2.535, p = 0.0163
대도시와 비대도시 간 상관관계에 유의미한 차이가 존재.


In [2]:
# 1. 데이터 불러오기
asthma_df = pd.read_excel('Asthma_top30_Sum_removed_outliers.xlsx', index_col=0, parse_dates=True)
dust_df = pd.read_excel("pm25_top30_fixed.xlsx", index_col=0, parse_dates=True)


# 2. 공통 날짜 추출
common_index = dust_df.index.intersection(asthma_df.index)
dust_df = dust_df.loc[common_index].sort_index()
asthma_df = asthma_df.loc[common_index].sort_index()

# # ✅ 3. 모든 값을 숫자로 변환하고 NaN 제거
# dust_df = dust_df.apply(pd.to_numeric, errors='coerce')
# asthma_df = asthma_df.apply(pd.to_numeric, errors='coerce')
# dust_df = dust_df.dropna()
# asthma_df = asthma_df.loc[dust_df.index]

# 3. 대도시 / 비대도시 분할
dust_major = dust_df.iloc[:, 0:8]
dust_minor = dust_df.iloc[:, 8:]

asthma_major = asthma_df.iloc[:, 0:8]
asthma_minor = asthma_df.iloc[:, 8:]

In [3]:
# 4. 상관계수 계산 함수
def get_corrs(df1, df2):
    return [pearsonr(df1.iloc[:, i], df2.iloc[:, i])[0] for i in range(df1.shape[1])]

major_corrs = get_corrs(dust_major, asthma_major)
minor_corrs = get_corrs(dust_minor, asthma_minor)


In [ ]:
# 평균 비교 및 t-검정
from scipy.stats import ttest_ind

t_stat, p_value = ttest_ind(major_corrs, minor_corrs, equal_var=False)

print("대도시 평균 상관계수:", np.mean(major_corrs))
print("비대도시 평균 상관계수:", np.mean(minor_corrs))
print("T-검정 결과: t =", t_stat, ", p =", p_value)

if p_value < 0.05:
    print("대도시와 비대도시 간 상관관계에 유의미한 차이가 존재.")
else:
    print("대도시와 비대도시 간 상관관계에 유의미한 차이 없음.")

대도시 평균 상관계수: 0.5200376216945806
비대도시 평균 상관계수: 0.4080278336906824
T-검정 결과: t = 2.612358305222492 , p = 0.022399502188818313
대도시와 비대도시 간 상관관계에 유의미한 차이가 존재.


In [9]:
# 초미세먼지 피셔 검정

import pandas as pd
import numpy as np
from scipy.stats import fisher_exact

# 1. 데이터 불러오기
asthma_df = pd.read_excel('Asthma_top30_Sum_removed_outliers.xlsx',
                          index_col=0, parse_dates=True)
dust_df   = pd.read_excel('pm25_top30_fixed.xlsx',
                          index_col=0, parse_dates=True)

# 2. 공통 날짜 동기화
common_idx = dust_df.index.intersection(asthma_df.index)
dust_df   = dust_df.loc[common_idx].sort_index()
asthma_df = asthma_df.loc[common_idx].sort_index()

# 3. 숫자 변환 및 결측치 제거
dust_df   = dust_df.apply(pd.to_numeric, errors='coerce').dropna()
asthma_df = asthma_df.apply(pd.to_numeric, errors='coerce').loc[dust_df.index]

# 4. 대도시 / 비대도시 분할 (0:8 대도시, 8: 끝 비대도시)
dust_major   = dust_df.iloc[:, :8]
dust_minor   = dust_df.iloc[:, 8:]
asthma_major = asthma_df.iloc[:, :8]
asthma_minor = asthma_df.iloc[:, 8:]

# 5. 평균 기준 이진화 (True/False)
dust_major_bin   = dust_major.gt(dust_major.mean())
dust_minor_bin   = dust_minor.gt(dust_minor.mean())
asthma_major_bin = asthma_major.gt(asthma_major.mean())
asthma_minor_bin = asthma_minor.gt(asthma_minor.mean())

# 6. Fisher 검정 함수
def fisher_test(df1_bin, df2_bin):
    p_vals = []
    for i in range(df1_bin.shape[1]):
        tbl = pd.crosstab(df1_bin.iloc[:, i],
                          df2_bin.iloc[:, i])
        if tbl.shape == (2, 2):
            _, p = fisher_exact(tbl)
        else:
            p = np.nan
        p_vals.append(p)
    return p_vals

# 7. 대도시/비대도시별 p-값 계산 및 출력
major_p = fisher_test(dust_major_bin, asthma_major_bin)
minor_p = fisher_test(dust_minor_bin, asthma_minor_bin)

print("대도시 Fisher 검정 p-값:", major_p)
print("비대도시 Fisher 검정 p-값:", minor_p)


대도시 Fisher 검정 p-값: [0.00021850265344794994, 1.0143227727644239e-07, 3.723670443379387e-05, 0.00024370342346980483, 7.071709378578791e-07, 5.446800981292854e-06, 1.4704148232517276e-06, 0.0041245867133721985]
비대도시 Fisher 검정 p-값: [0.0118543319994536, 0.0009841241154624105, 0.0004940942747315203, 0.0035921872197405033, 0.1191863680139637, 0.007620343007956382, 0.00044623939409026636, 0.01194052194499604, 0.0036453481924410116, 0.05738059149537865, 1.543947909548943e-08, 0.001057715449011629, 9.19838008201434e-05, 1.6293411112769097e-10, 2.4191288556393257e-06, 0.0035992675654283966, 0.0004559567688786165, 0.05341892878109237, 0.019891992319007113, 0.00045604629515997436, 0.00022048067034118765, 5.8968831322919295e-06, 0.003508653376333956, 1.1030535877585584e-05, 0.020059041587126854, 0.05330732294512777]
